In [8]:
! curl -O https://physionet.org/static/published-projects/mimiciii-demo/mimic-iii-clinical-database-demo-1.4.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.8M  100 10.8M    0     0  1252k      0  0:00:08  0:00:08 --:--:-- 1270k


In [11]:
! unzip mimic-iii-clinical-database-demo-1.4.zip && mv mimic-iii-clinical-database-demo-1.4 mimic

Archive:  mimic-iii-clinical-database-demo-1.4.zip
   creating: mimic-iii-clinical-database-demo-1.4/
  inflating: mimic-iii-clinical-database-demo-1.4/ADMISSIONS.csv  
  inflating: mimic-iii-clinical-database-demo-1.4/CALLOUT.csv  
  inflating: mimic-iii-clinical-database-demo-1.4/CAREGIVERS.csv  
  inflating: mimic-iii-clinical-database-demo-1.4/CHARTEVENTS.csv  
  inflating: mimic-iii-clinical-database-demo-1.4/CPTEVENTS.csv  
  inflating: mimic-iii-clinical-database-demo-1.4/DATETIMEEVENTS.csv  
  inflating: mimic-iii-clinical-database-demo-1.4/D_CPT.csv  
  inflating: mimic-iii-clinical-database-demo-1.4/DIAGNOSES_ICD.csv  
  inflating: mimic-iii-clinical-database-demo-1.4/D_ICD_DIAGNOSES.csv  
  inflating: mimic-iii-clinical-database-demo-1.4/D_ICD_PROCEDURES.csv  
  inflating: mimic-iii-clinical-database-demo-1.4/D_ITEMS.csv  
  inflating: mimic-iii-clinical-database-demo-1.4/D_LABITEMS.csv  
  inflating: mimic-iii-clinical-database-demo-1.4/DRGCODES.csv  
  inflating: mimic-iii

## 1. Data Loading

In [23]:
from cardea.data_assembling import load_mimic_data

subset = [
    "ADMISSIONS",
    "CALLOUT",
    "CAREGIVERS",
    "CHARTEVENTS",
    "PATIENTS",
    "PRESCRIPTIONS"
]

subset = [x.lower() for x in subset]

es = load_mimic_data(path="mimic/", subset=subset)
es

/Users/sarah/anaconda3/envs/cardea/lib/python3.7/site-packages/ipykernel_launcher.py:14: DtypeWarning: Columns (8,10,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  


Entityset: mimic
  Entities:
    admissions [Rows: 129, Columns: 19]
    callout [Rows: 77, Columns: 24]
    caregivers [Rows: 7567, Columns: 4]
    chartevents [Rows: 758355, Columns: 15]
    patients [Rows: 100, Columns: 8]
    prescriptions [Rows: 10398, Columns: 19]
  Relationships:
    callout.hadm_id -> admissions.hadm_id
    chartevents.hadm_id -> admissions.hadm_id
    prescriptions.hadm_id -> admissions.hadm_id
    chartevents.cgid -> caregivers.cgid
    admissions.subject_id -> patients.subject_id
    callout.subject_id -> patients.subject_id
    chartevents.subject_id -> patients.subject_id
    prescriptions.subject_id -> patients.subject_id

## 2. Prediction Problem

In [42]:
from functools import partial

from cardea.data_labeling import DataLabeler
from cardea.data_labeling.length_of_stay import length_of_stay

los_7d = partial(length_of_stay, k=7)

labeler = DataLabeler(los_7d)

In [43]:
target_times, pred_type, meta = labeler.generate_label_times(es, verbose=True)

Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| hadm_id: 129/129 


In [44]:
target_times.head()

,hadm_id,time,label
0,100375,2129-05-02 00:12:00,False
1,100969,2142-11-26 21:20:00,False
2,101361,2145-12-01 18:13:00,True
3,102203,2127-07-23 15:21:00,True
4,103379,2170-12-15 03:14:00,True


## 3.  Featurizing

In [45]:
from cardea.featurizing import Featurization

ft = Featurization()

In [46]:
target = meta.get('entity')

fm, fm_definition = ft.generate_feature_matrix(es, target, target_times, verbose=True)

Built 338 features
Elapsed: 00:00 | Progress:   1%|██                                                                                                                                                                                                                                              

/Users/sarah/anaconda3/envs/cardea/lib/python3.7/site-packages/pandas/core/frame.py:4486: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[k] = com.apply_if_callable(v, data)
/Users/sarah/anaconda3/envs/cardea/lib/python3.7/site-packages/pandas/core/frame.py:4486: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[k] = com.apply_if_callable(v, data)


Elapsed: 00:00 | Progress:   2%|█████▊                                                                                                                                                                                                                                          

/Users/sarah/anaconda3/envs/cardea/lib/python3.7/site-packages/pandas/core/frame.py:4486: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[k] = com.apply_if_callable(v, data)
/Users/sarah/anaconda3/envs/cardea/lib/python3.7/site-packages/pandas/core/frame.py:4486: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[k] = com.apply_if_callable(v, data)


Elapsed: 00:01 | Progress:   4%|██████████▎                                                                                                                                                                                                                                     

...

Elapsed: 00:29 | Progress:  93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏               

/Users/sarah/anaconda3/envs/cardea/lib/python3.7/site-packages/pandas/core/frame.py:4486: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[k] = com.apply_if_callable(v, data)
/Users/sarah/anaconda3/envs/cardea/lib/python3.7/site-packages/pandas/core/frame.py:4486: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[k] = com.apply_if_callable(v, data)


Elapsed: 00:29 | Progress:  95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋            

/Users/sarah/anaconda3/envs/cardea/lib/python3.7/site-packages/pandas/core/frame.py:4486: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[k] = com.apply_if_callable(v, data)


Elapsed: 00:30 | Progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████


In [47]:
fm.head()

,row_id,subject_id,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,...,NUM_UNIQUE(prescriptions.patients.gender),PERCENT_TRUE(prescriptions.IS_WEEKEND(enddate)),PERCENT_TRUE(prescriptions.IS_WEEKEND(startdate)),SKEW(prescriptions.patients.expire_flag),SKEW(prescriptions.patients.row_id),STD(prescriptions.patients.expire_flag),STD(prescriptions.patients.row_id),SUM(prescriptions.patients.expire_flag),SUM(prescriptions.patients.row_id),label
hadm_id,,,,,,,,,,,,,,,,,,,,,
100375,12305,10056,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SNF,Medicare,NaN,CHRISTIAN SCIENTIST,UNKNOWN (DEFAULT),WHITE,...,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,21.0,199794.0,False
100969,40554,42430,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CHRISTIAN SCIENTIST,NaN,WHITE,...,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,15.0,471435.0,False
101361,40379,41914,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,...,1.0,0.636364,0.0,0.0,0.0,0.0,0.0,11.0,344300.0,True
102203,40462,42135,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,ENGL,MUSLIM,MARRIED,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,...,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,21.0,658350.0,True
103379,41092,44228,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,...,1.0,0.473684,1.0,0.0,0.0,0.0,0.0,57.0,1816704.0,True


## 4. Modeling

In [48]:
from sklearn.model_selection import train_test_split

y = fm.pop('label').values
X = fm.values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=0)

In [49]:
from cardea.modeling import Modeler

pipeline = "XGB"

model = Modeler(pipeline, pred_type)

In [50]:
model.fit(X_train, y_train)

In [51]:
pred = model.predict(X_test)

In [52]:
pred

array([ True, False, False, False, False, False,  True, False, False,
       False, False,  True, False, False,  True, False, False, False,
        True,  True, False, False, False, False, False, False])